<a href="https://colab.research.google.com/github/pablobermudez/Up/blob/main/HPC/Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El ejercicio posee dos versiones del algoritmo de ordenamiento denominado **Bubble Sort**, la primera de ellas deberá ejecutarse en el entorno CPU con el lenguaje Python y la segunda en el entorno GPU utilizando el lenguaje Cuda junto con la librería pycuda.

La finalidad del ejercicio no es centrarse en la complejidad del algoritmo utilizado, sino en la información que nos provee y en el sustento teórico que poseen y en las diferencias que presenta el mismo al ejecutarse en entorno **CPU** y **GPU**.

Se medirán los tiempos que demora la ejecución en ambos ambientes, y se hará foco en porque es necesario modificar sus implementaciones más allá del lenguaje de programación utilizado.

Para entender mejor lo mencionado, ejecutaremos las pruebas y veremos entonces las conclusiones.





#2 Armado del ambiente
Instalar en el cuaderno el módulo CUDA de Python.


In [ ]:
!pip install pycuda

#3 Desarrollo

##3.1 Desarrollo - CPU



In [ ]:
# --------------------------------------------
#@title 3.1.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   0#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (cantidad_N <= 0):
  raise Exception("Solo se aceptan números positivos") 
# --------------------------------------------

import random
import numpy as np

from datetime import datetime
tiempo_total = datetime.now()


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# --------------------------------------------
# Defino variables
arr = [random.randint(1,10) for _ in range(cantidad_N)]


# Defino algoritmo
def bubbleSort(arr): 
    n = len(arr) 
    for i in range(n-1): 
        for j in range(0, n-i-1): 
            if arr[j] > arr[j+1] : 
                arr[j], arr[j+1] = arr[j+1], arr[j] 

# Ejecuto ordenamiento
bubbleSort(arr) 


# CPU - Informo tiempos, hilos y bloques.
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )

# CPU - Informo el resutlado.
print( "------------------------------------")
print( "Array: " )
print( arr )



##![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Importante:** Debe cambiar de entorno de ejecución a GPU para poder ejecutar el siguiente desarrollo.

##3.2 Desarrollo - GPU


In [ ]:
# --------------------------------------------
#@title 3.2.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   300#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (cantidad_N <= 0):
  raise Exception("Solo se aceptan números positivos") 
# --------------------------------------------

import sys
import random
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

from datetime import datetime
tiempo_total = datetime.now()




# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# CPU - Defino la memoria de los vectores en cpu.
x_cpu = [random.randint(1,10) for _ in range(cantidad_N)]
x_cpu = np.array(x_cpu, dtype=np.int32)
r_cpu = np.empty_like( x_cpu )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""

#include <stdio.h>
__global__ void bubble_sort_gpu( int n, float *X)
{

   int i = threadIdx.x + blockDim.x * blockIdx.x;
   float aux;
    for (int p = 0; p < n; ++p) {
        if ((i - p) % 2 && i < n - 1 && X[i] > X[i + 1]) {
            aux = X[i];
            X[i] = X[i + 1];
            X[i + 1] = aux;
        }     
        __syncthreads();
    }
}

""") 
# CPU - Genero la función kernel.
kernel = module.get_function("bubble_sort_gpu")
tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = np.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

kernel( np.int32(cantidad_N), x_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, x_gpu )

tiempo_total = datetime.now() - tiempo_total

# TODO - Informo tiempos, hilos y bloques.
print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )

# 4 Tabla de pasos

##4.1 Tabla de Pasos - CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  random.randint(1,10) for _ in range(cantidad_N) | Inicializa el vector **arr** con cantidad_N de números random entre el 1 y el 10.
CPU      |  bubbleSort(arr)      | Ordena el vector con burbujeo.
CPU      |  print()               | Informo los resultados.

##4.2 Tabla de Pasos - GPU

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  random.randint(1,10) for _ in range(cantidad_N) | Inicializa el vector **x_cpu** con cantidad_N de números random entre el 1 y el 10.
CPU      |  np.array()            | Defino los valores dentro del array **x_cpu** como int32.
CPU      |  np.empty_like( **x_cpu** ) | Genera un array vacio del mismo tipo que **x_cpu** y se lo asigna a **r_cpu**.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
**GPU**  |  __syncthreads()       | Sincroniza los hilos de los distintos bloques para que puedan realizar la tarea de ordenamiento correctamente.
CPU      |  SourceModule()        | Define el código del kernel. 
CPU      |  module.get_function() | Genera la función del kernel GPU.
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

Notamos al ejecutar ambas pruebas, que cuando nuestra variable **cantidad_N** corresponde a valores numéricos grandes, la implementación realizada en conjunto con el procesador GPU da una clara ventaja utilizando su paralelismo y logra tiempos mucho más cortos para realizar el ordenamiento.

Este hecho, no implica que el Bubble Sort sea un algoritmo ideal para delegarlo al GPU ya que este deberá realizar muchas operaciones extras para poder sincronizar los hilos ejecutados y esto llevará un tiempo considerable.[2] Un algoritmo de ordenamiento mas idóneo para realizar con ayuda del GPU sería algún algoritmo de ordenamiento del tipo "divide y vencerás" cuya naturaleza le permite no tener que forzar la sincronización de todos los hilos ejecutados.

[1]La complejidad computacional del algoritmo Bubble Sort en el peor de los casos corresponde a O(n*n) ejecutándose en CPU, mientras que se considera que, si el mismo es ejecutado con ayuda del GPU, esta complejidad computacional disminuye considerablemente al orden de O(n*n/p) siendo p el número de hilos ejecutados.

Como posibilidad de expandir este ejercicio, se podría implementar en paralelo otro algoritmo de ordanmiento del tipo QuickSort o MergeSort para hacer más notorias las diferencias de implementación con el framework pycuda.


# 6 Referencias


*   [1] https://dergipark.org.tr/en/download/article-file/225714
*   [2] https://arxiv.org/ftp/arxiv/papers/1505/1505.07605.pdf
